# Example 01: WS1S Logic
In this example, we will demonstrate the capabilities of the `libmata` library on usage for deciding WS1S logic. WS1S is a weak monadic second order logic with one sucessor, which means it can quantify over sets and structure with one successor, making it suitable for modelling and reasoning about linear structures.

In particular, we will exploit the correspondence between the formulae of the WS1S logic and finite automata. For given formula we will show we can construct an automaton, whose language we can further analyse to decide whether the formula is: (1) satisfiable, i.e., for some words true, for some false; (2) unsatisfiable, i.e., always false; or (3) valid, i.e., always true.

In this example, we will limit ourselves to first order variables, such as `x` or `y`. We will encode them as strings, of `0` and `1`, such that the index of the first `1` will correspond to the numerical value of the variable. E.g. the word `0 0 1` corresponds to `2`. For each variable we will have one such column in the string. E.g. we will encode that `x = 3` and `y = 1` as word `00 01 00 10`: the first column corresponds to variable `x` and the second to variable `y`.

#### Example formula
In this example, we will analyze the decidability of the following formula: $(x = y + 1 \wedge y = z + 1) \rightarrow x = z + 2$.

The semantics of the binary operator `+` is as expected.

In [1]:
import mata
config = mata.store()
config['alphabet'] = mata.OnTheFlyAlphabet()

#### Atomic formulae
First, we will construct automata corresponding to atomic formulae. Since, we have three variables, in each symbol we have three values. Note, that since, we do not use the variable `z` in the first automaton, we "don't care" about its value, and hence we add two transitions, one for `z = 0` and one for `z = 1`. In decision procedures this is usually implemented in much more efficient way.

In [2]:
x_is_y_plus_one = mata.Nfa(3, label="x = y + 1")
x_is_y_plus_one.make_initial_state(0)
x_is_y_plus_one.add_transition(0, "000", 0)
x_is_y_plus_one.add_transition(0, "001", 0)
x_is_y_plus_one.add_transition(0, "010", 1)
x_is_y_plus_one.add_transition(0, "011", 1)
x_is_y_plus_one.add_transition(1, "100", 2)
x_is_y_plus_one.add_transition(1, "101", 2)
x_is_y_plus_one.add_transition(2, "000", 2)
x_is_y_plus_one.add_transition(2, "001", 2)
x_is_y_plus_one.make_final_state(2)

y_is_z_plus_one = mata.Nfa(3, label="y = z + 1")
y_is_z_plus_one.make_initial_state(0)
y_is_z_plus_one.add_transition(0, "000", 0)
y_is_z_plus_one.add_transition(0, "100", 0)
y_is_z_plus_one.add_transition(0, "001", 1)
y_is_z_plus_one.add_transition(0, "101", 1)
y_is_z_plus_one.add_transition(1, "010", 2)
y_is_z_plus_one.add_transition(1, "110", 2)
y_is_z_plus_one.add_transition(2, "000", 2)
y_is_z_plus_one.add_transition(2, "100", 2)
y_is_z_plus_one.make_final_state(2)

x_is_z_plus_two = mata.Nfa(4, label="x = z + 2")
x_is_z_plus_two.make_initial_state(0)
x_is_z_plus_two.add_transition(0, "000", 0)
x_is_z_plus_two.add_transition(0, "010", 0)
x_is_z_plus_two.add_transition(0, "001", 1)
x_is_z_plus_two.add_transition(0, "011", 1)
x_is_z_plus_two.add_transition(1, "000", 2)
x_is_z_plus_two.add_transition(1, "010", 2)
x_is_z_plus_two.add_transition(2, "100", 3)
x_is_z_plus_two.add_transition(2, "110", 3)
x_is_z_plus_two.add_transition(3, "000", 3)
x_is_z_plus_two.add_transition(3, "010", 3)
x_is_z_plus_two.make_final_state(3)

We will plot all of the resulting automata using the `mata.plot` function

In [3]:
mata.plot(x_is_y_plus_one, y_is_z_plus_one, x_is_z_plus_two)

#### Logical connectives

By induction on the structure of the formulae we construct the more complex automata. First, we will construct the automaton corresponding to the premise of the implication. The logical `and` corresponds to the intersection of the automata.

In [4]:
premise = mata.Nfa.intersection(x_is_y_plus_one, y_is_z_plus_one)
premise.label = "x = y + 1 ∧ y = z + 1"
mata.plot(premise)

Since, there is no corresponding operation for an logical implication, we use one of the laws to simplify the implication $A \rightarrow B$ to $\neg A \vee B$

Hence, we next complement the premise automaton. Note, that we need to supply the alphabet to perform the complementation. Since, we have not set any alphabet for this example, a default alphabet is used, that maps the seen strings to unique identifications.

In [5]:
not_premise = mata.Nfa.complement(premise, alphabet=config['alphabet'])
not_premise.label = "¬(x = y + 1 ∧ y = z + 1)"
mata.plot(not_premise)

Finally, we do the union of the complemented premise with the last atomic automaton. We further determinize the result to obtain complete and deterministic automaton.

In [6]:
formula = mata.Nfa.determinize(mata.Nfa.union(not_premise, x_is_z_plus_two))
formula.label = "¬(x = y + 1 ∧ y = z + 1) ∨ x = z + 2"
mata.plot(formula)

#### Minimization of the result
However, the resulting automaton is too complex. Hence, we try to minimize the results to merge equivalent (from the language perspective) states.

In [7]:
minimized_formula = mata.Nfa.minimize(formula)
minimized_formula.label = "¬(x = y + 1 ∧ y = z + 1) ∨ x = z + 2"
mata.plot(minimized_formula)

#### The final result
One can see that the resulting automaton is universal. I.e., it accepts everyting. This means, that the original formula $(x = y + 1 \wedge y = z + 1) \rightarrow x = z + 2$ is tautology.

We can verify our observation using the `is_universal` test implemented in the `libmata`

In [8]:
mata.Nfa.is_universal(minimized_formula, config['alphabet'])

True